In [139]:
# Request html of Durham properties
import requests
import re
from bs4 import BeautifulSoup
durham = "https://www.rightmove.co.uk/student-accommodation/Durham.html"
durhamListings = requests.get(durham)

In [140]:
# function that generates a list of hyperlinks for each property found
links = []
def getPropertyLinks():
    x = 0
    soup = BeautifulSoup(durhamListings.content,"html.parser")
    for a in soup.find_all("a", class_="propertyCard-link"):
        if a['href'] != "":
            hyperlink = "https://www.rightmove.co.uk" + a['href']
            if hyperlink not in links:
                links.append(hyperlink)
                x += 1
    print(x, "properties found")
    print(links)

In [141]:
getPropertyLinks()

24 properties found
['https://www.rightmove.co.uk/properties/120163952#/?channel=STU_LET', 'https://www.rightmove.co.uk/properties/120013298#/?channel=STU_LET', 'https://www.rightmove.co.uk/properties/119851841#/?channel=STU_LET', 'https://www.rightmove.co.uk/properties/119658029#/?channel=STU_LET', 'https://www.rightmove.co.uk/properties/119443010#/?channel=STU_LET', 'https://www.rightmove.co.uk/properties/83598762#/?channel=STU_LET', 'https://www.rightmove.co.uk/properties/119186702#/?channel=STU_LET', 'https://www.rightmove.co.uk/properties/119092295#/?channel=STU_LET', 'https://www.rightmove.co.uk/properties/118728917#/?channel=STU_LET', 'https://www.rightmove.co.uk/properties/116883431#/?channel=STU_LET', 'https://www.rightmove.co.uk/properties/116218499#/?channel=STU_LET', 'https://www.rightmove.co.uk/properties/115914395#/?channel=STU_LET', 'https://www.rightmove.co.uk/properties/115900127#/?channel=STU_LET', 'https://www.rightmove.co.uk/properties/115823618#/?channel=STU_LET', 

In [142]:
# define property class with attributes
properties = []
class Property:
    def __init__(self, name, price, propertytype, beds, baths, link, furnished, billsincluded, parkingspace):
        self.name = name
        self.price = price
        self.propertytype = propertytype
        self.beds = beds
        self.baths = baths
        self.link = link
        self.furnished = furnished
        self.billsincluded = billsincluded
        self.parkingspace = parkingspace
        

In [143]:
def findKeyInfo():
    propertytype = "unknown"
    beds = "unknown"
    baths = "unknown"
    for m in broth.find_all('div', class_='_1u12RxIYGx3c84eaGxI6_b'):
        for mm in m.find_all('div', class_='tmJOVKTrHAB4bLpcMjzQ'):
            if mm.text == "PROPERTY TYPE":
                for mmm in m.find_all('div', class_='_1fcftXUEbWfJOJzIUeIHKt'):
                    propertytype = mmm.text
            if mm.text == "BEDROOMS":
                for mmm in m.find_all('div', class_='_1fcftXUEbWfJOJzIUeIHKt'):
                    beds = mmm.text
            if mm.text == "BATHROOMS":
                for mmm in m.find_all('div', class_='_1fcftXUEbWfJOJzIUeIHKt'):
                    baths = mmm.text

In [144]:
def findBillArrangement():
# check key features first
    ul = broth.find_all('ul', class_="_1uI3IvdF5sIuBtRIvKrreQ")
    for li in ul:
        liF = li.find_all('li')
        for li in liF:
            string = li.text
            checkBills = re.search("(?i)(\\bbills\\b)", string)
            if checkBills:
                checkNot = re.search("(?i)(\\bnot\\b)", string)
                checkNo = re.search("(?i)(\\bno\\b)", string)
                checkNoninclusive = re.search("(?i)(\\bnon-inclusive\\b)", string)
                checkExclusive = re.search("(?i)(\\bexclusive\\b)", string)
                checkOption = re.search("(?i)(\\boption\\b)", string)
                if checkNot:
                    billsincluded = False
                elif checkNo:
                    billsincluded = False
                elif checkNoninclusive:
                    billsincluded = False
                elif checkExclusive:
                    billsincluded = False
                elif checkOption:
                    billsincluded = "Option Available"
                else:
                    billsincluded = True
                return billsincluded

In [145]:
# function that takes a Rightmove property listing as input, scrapes property data, then appends to list of properties

def scrapePropertyLink(propertyLink):
    page = requests.get(propertyLink)
    broth = BeautifulSoup(page.content,"html.parser")

    # find house name
    for h1 in broth.find_all('h1', itemprop='streetAddress'):
        name = h1.text

    # find price per week
    for div in broth.find_all('div', class_="HXfWxKgwCdWTESd5VaU73"):
        price = div.text
        
    # find house type, no. of beds and bathrooms
    findKeyInfo()

    # find hyperlink to property
    link = propertyLink
    
    # furnishing status
    furnished = "Unknown"
    div = broth.find_all('div', class_="_2RnXSVJcWbWv4IpBC1Sng6")
    for dd in div:
        ddF = dd.find_all('dd')
        for dd in ddF:
            if dd.text == "Furnished":
                furnished = True
            if dd.text == "Unfurnished":
                furnished = False

    # bills
    findBillArrangement()
    
    # parking
    parkingspace = "Unknown"
    
    # Create object with known information, then append to list of properties
    prop = Property(name, price, propertytype, beds, baths, link, furnished, billsincluded, parkingspace)
    
    properties.append(prop)

In [146]:
properties = []
current = "https://www.rightmove.co.uk/properties/120013298#/?channel=STU_LET"
scrapePropertyLink(current)

In [147]:
def showProperty(g):
    print("Name:", properties[g].name)
    print("Price:", properties[g].price)
    print("Type:", properties[g].propertytype)
    print("Bedrooms:", properties[g].beds)
    print("Bathrooms:", properties[g].baths)
    print("Link:", properties[g].link)
    print("Furnished:", properties[g].furnished)
    print("Bills Included:", properties[g].billsincluded)
    print("Parking Space:", properties[g].parkingspace)

In [148]:
showProperty(0)

Name: Flat C, Brookwell House, Durham, DH1
Price: £133 pw
Type: unknown
Bedrooms: ×5
Bathrooms: ×1
Link: https://www.rightmove.co.uk/properties/120013298#/?channel=STU_LET
Furnished: True
Bills Included: True
Parking Space: Unknown
